In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# --- Cell 1: Environment Setup & Reset ---

!rm -rf /kaggle/working/dendPLRNN
!git clone --depth 1 https://github.com/DurstewitzLab/dendPLRNN.git /kaggle/working/dendPLRNN
print("✅ Repository cloned.")

# Install dependencies
!pip install numpy scipy scikit-learn matplotlib tensorboardX
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
print("✅ Dependencies installed.")

# Ensure Python can find project modules
import sys
sys.path.insert(0, "/kaggle/working/dendPLRNN/BPTT_TF")
sys.path.insert(0, "/kaggle/working/dendPLRNN")
print("✅ PYTHONPATH ready.")

# Change to the correct directory
%cd /kaggle/working/dendPLRNN/BPTT_TF
print(f"✅ Current directory: $(pwd)")

# Fix known import and experiment issues
!sed -i "s/from pandas.core.indexes import numeric/from pandas.api.types import is_numeric_dtype as numeric/" main_eval.py
!sed -i "s/Argument('n_epochs', \[[0-9]*\])/Argument('n_epochs', [10])/" \
/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG/ubermain.py
print("✅ Pandas and Epoch fixes applied.")

!nvidia-smi

Cloning into '/kaggle/working/dendPLRNN'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 137 (delta 27), reused 122 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (137/137), 60.51 MiB | 40.87 MiB/s, done.
Resolving deltas: 100% (27/27), done.
✅ Repository cloned.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━

In [2]:
# --- Cell 2 (CORRECTED): Python-based File Modification ---

import os
import textwrap # <-- This import fixes the indentation

# --- 1. Define the "Novelty" Module Code ---
novelty_code = """
\"\"\"
Minimal, defensive novelty utilities for PLRNN training.
Keeps logic isolated to avoid repeated in-place edits of big files.
\"\"\"

import torch as tc
import torch.nn as nn

def inject_latent_noise(x, training=True, noise_std=0.02):
    \"\"\"Add Gaussian noise to input tensor x if training is True.\"\"\"
    if not training:
        return x
    try:
        return x + tc.randn_like(x) * float(noise_std)
    except Exception:
        return x

def manifold_attractor_regularization(model, lambda_mar=0.01, frac=0.2):
    \"\"\"
    Minimal manifold-attractor regularizer.
    - If model has attribute 'A' (tensor or parameter), compute small penalty.
    - Returns a scalar tensor on same device; returns 0.0 tensor if not applicable.
    \"\"\"
    # safe guards
    try:
        if not hasattr(model, "A"):
            return tc.tensor(0.0, device=next(model.parameters()).device)
        A = getattr(model, "A")
        # convert to tensor if Parameter
        if isinstance(A, nn.Parameter):
            A = A.data
        A = tc.as_tensor(A)
        device = A.device
        W = getattr(model, "W", None)
        h = getattr(model, "h", None)
        M = A.shape[0]
        Mreg = max(1, int(M * float(frac)))
        A_diag = tc.diag(A)
        mar_loss = tc.tensor(0.0, device=device)
        for i in range(Mreg):
            mar_loss = mar_loss + (A_diag[i] - 1.0) ** 2
            if h is not None:
                try:
                    mar_loss = mar_loss + (tc.as_tensor(h[i]) ** 2)
                except Exception:
                    pass
            if W is not None:
                try:
                    row = tc.as_tensor(W[i, :])
                    mar_loss = mar_loss + (tc.sum(row ** 2) - row[i] ** 2)
                except Exception:
                    pass
        return float(lambda_mar) * mar_loss
    except Exception:
        # On any unexpected issue, return zero tensor safely
        try:
            return tc.tensor(0.0, device=next(model.parameters()).device)
        except Exception:
            return tc.tensor(0.0)

def temporal_self_distillation_from_model(model, weight=0.05):
    \"\"\"
    Try to extract a latent trajectory from the model and compute MSE between adjacent timesteps.
    Looks for common attributes (Z, last_z, z_hist). If none found, returns 0.
    \"\"\"
    try:
        z = None
        for attr in ("Z", "z_hist", "last_z", "Z_hist"):
            if hasattr(model, attr):
                z = getattr(model, attr)
                break
        # if z is a list or tuple, coerce to tensor if possible
        if z is None:
            return tc.tensor(0.0, device=next(model.parameters()).device)
        
        # Ensure z is a tensor
        if not isinstance(z, tc.Tensor):
            # Try to convert list of tensors, e.g., from a .cpu() loop
            if isinstance(z, (list, tuple)) and len(z) > 1 and isinstance(z[0], tc.Tensor):
                 zt = tc.stack(z)
            else:
                 zt = tc.as_tensor(z)
        else:
            zt = z

        if zt.dim() < 2 or zt.shape[0] < 2:
            return tc.tensor(0.0, device=zt.device)
        diff = zt[1:] - zt[:-1].detach()
        return float(weight) * tc.mean(diff ** 2)
    except Exception:
        try:
            return tc.tensor(0.0, device=next(model.parameters()).device)
        except Exception:
            return tc.tensor(0.0)

def init_dendritic_if_possible(model, init_scale=0.05):
    \"\"\"
    If model has W and supports register_parameter, create and register 'U' parameter.
    Safe no-op if not possible.
    \"\"\"
    try:
        if hasattr(model, "W") and hasattr(model, "register_parameter") and not hasattr(model, "U"):
            W = getattr(model, "W")
            # create a parameter of same shape as W
            U = nn.Parameter(tc.randn_like(tc.as_tensor(W)) * float(init_scale))
            model.register_parameter("U", U)
            return True
    except Exception:
        pass
    return False

def apply_dendritic_gate(z, model):
    \"\"\"
    Apply gating if model has parameter U; else return z unchanged.
    \"\"\"
    try:
        if hasattr(model, "U"):
            U = getattr(model, "U")
            # Apply dendritic-style gating
            return tc.sigmoid(tc.matmul(z, U)) * tc.relu(z)
    except Exception:
        pass
    # Fallback to standard relu if no U or if error
    return tc.relu(z)
"""

# --- 2. Define the Code to Inject (DEDENTED) ---
# This block is wrapped in textwrap.dedent()
# This is the key fix that corrects the indentation.
insertion_block = textwrap.dedent("""
    # --- novelties (MAR / TSD / safe noise) ---
    try:
        # manifold-attractor regularization (safe)
        mar = novelties.manifold_attractor_regularization(self.model, lambda_mar=0.01)
        if isinstance(mar, float) or (hasattr(mar, "item") and callable(getattr(mar, "item"))):
            loss = loss + (mar if isinstance(mar, float) else mar)
    except Exception:
        pass
    try:
        # temporal self-distillation (if model exposes latent trajectory)
        tsd = novelties.temporal_self_distillation_from_model(self.model, weight=0.05)
        if hasattr(tsd, "item"):
            loss = loss + tsd
    except Exception:
        pass
    try:
        # add an extra small noise via helper (this is optional and safe)
        if "inp" in locals():
            inp = novelties.inject_latent_noise(inp, training=self.model.training if hasattr(self.model, "training") else True, noise_level=getattr(self, "noise_level", 0.02))
    except Exception:
        pass
""")

# Define file paths
novelties_file = "bptt/novelties.py"
algo_file = "bptt/bptt_algorithm.py"

# --- 3. Write the new novelties.py file ---
try:
    with open(novelties_file, "w") as f:
        f.write(novelty_code)
    print(f"✅ Successfully created {novelties_file}")

    # --- 4. Patch bptt_algorithm.py ---
    
    # Add the import at the top
    with open(algo_file, "r") as f:
        content = f.read()
    
    if "from bptt import novelties" not in content:
        content = "from bptt import novelties\n" + content
        with open(algo_file, "w") as f:
            f.write(content)
        print(f"✅ Added import to {algo_file}")
    
    # Insert the loss block
    with open(algo_file, "r") as f:
        lines = f.readlines()

    new_lines = []
    inserted = False
    target_line = "loss = self.compute_loss(pred, target)"

    for line in lines:
        new_lines.append(line)
        # Check if this is the target line and we haven't inserted yet
        if target_line in line and not inserted:
            # Find the indentation of the target line
            indentation = line[:len(line) - len(line.lstrip())]
            # Add the insertion block with the same indentation
            indented_insertion = "\n".join([f"{indentation}{l}" for l in insertion_block.splitlines() if l])
            new_lines.append(indented_insertion + "\n")
            inserted = True
            print(f"✅ Successfully inserted novelty block into {algo_file}")

    if not inserted:
        print(f"⚠️ WARNING: Could not find target line '{target_line}' in {algo_file}. Patch failed.")
    else:
        # Write the modified content back
        with open(algo_file, "w") as f:
            f.writelines(new_lines)
        print("✅ File patching complete.")

except Exception as e:
    print(f"❌ An error occurred during file modification: {e}")

✅ Successfully created bptt/novelties.py
✅ Added import to bptt/bptt_algorithm.py
✅ Successfully inserted novelty block into bptt/bptt_algorithm.py
✅ File patching complete.


In [4]:
# --- Cell 3: Syntax Check and Run Training ---

!echo "Running python syntax check..."
!python -m py_compile bptt/novelties.py
!python -m py_compile bptt/bptt_algorithm.py
!echo "✅ Syntax OK."



Running python syntax check...
✅ Syntax OK.


In [5]:
# --- GPU Diagnostic Cell ---
import torch

print(f"PyTorch Version: {torch.__version__}")
print("---")
print(f"Is CUDA (GPU) available? ==> {torch.cuda.is_available()}")
print("---")

if torch.cuda.is_available():
    print(f"Current GPU Name: {torch.cuda.get_device_name(0)}")
else:
    print("WARNING: PyTorch cannot find the GPU.")
    print("Please go to 'Settings' on the right and ensure 'Accelerator' is set to 'GPU'.")

print("\n--- nvidia-smi (Hardware Check) ---")
!nvidia-smi

PyTorch Version: 2.6.0+cu124
---
Is CUDA (GPU) available? ==> True
---
Current GPU Name: Tesla P100-PCIE-16GB

--- nvidia-smi (Hardware Check) ---
Wed Nov 12 14:38:38 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P0             27W /  250W |       3MiB /  16384MiB |      0%      Def

In [9]:
# --- Check Command: Add a "print" statement ---

!sed -i "/def manifold_attractor_regularization/a \    print('--- NOVELTY CODE IS RUNNING (MAR) ---', flush=True)" \
    /kaggle/working/dendPLRNN/BPTT_TF/bptt/novelties.py

print("✅ 'print' statement has been injected into 'bptt/novelties.py'.")
print("   Ready for final verification.")

✅ 'print' statement has been injected into 'bptt/novelties.py'.
   Ready for final verification.


In [13]:
!grep -n "numeric" main_eval.py


3:from pandas.api.types import is_numeric_dtype as numeric
260:        mse5 = (df.mean(0, numeric_only=True)['5'], df.std(numeric_only=True)['5'])
261:        mse10 = (df.mean(0, numeric_only=True)['10'], df.std(numeric_only=True)['10'])
262:        mse20 = (df.mean(0, numeric_only=True)['20'], df.std(numeric_only=True)['20'])
272:        pse = (df.mean(0, numeric_only=True)['mean'], df.std(numeric_only=True)['mean'])


In [14]:
%cd /kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG


/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG


In [15]:
!sed -i "s/Argument('use_gpu', \[[0-9]\+\])/Argument('use_gpu', [1])/" ubermain.py


In [16]:
!grep -n "use_gpu" ubermain.py


11:    When using GPU for training (i.e. Argument 'use_gpu 1')  it is generally
17:    args.append(Argument('use_gpu', [1])) # may wanna use gpu here


In [17]:
!sed -i "s/n_runs = [0-9]\+/n_runs = 1/" ubermain.py


In [18]:
!grep -n "n_runs" ubermain.py


4:def ubermain(n_runs):
28:    args.append(Argument('run', list(range(1, 1 + n_runs))))
36:    n_runs = 1
42:    args = ubermain(n_runs)


In [20]:
# --- Cell to Clean Up the Log ---
# This finds our print statement in novelties.py and deletes it.

!sed -i "/--- NOVELTY CODE IS RUNNING (MAR) ---/d" /kaggle/working/dendPLRNN/BPTT_TF/bptt/novelties.py

print("✅ 'novelties.py' has been cleaned.")
print("You are now ready to run your final training with a clean log.")

✅ 'novelties.py' has been cleaned.
You are now ready to run your final training with a clean log.


In [21]:
%cd /kaggle/working/dendPLRNN/BPTT_TF
!PYTHONPATH="/kaggle/working/dendPLRNN/BPTT_TF:/kaggle/working/dendPLRNN" \
python Experiments/Table1/ECG/ubermain.py


/kaggle/working/dendPLRNN/BPTT_TF
'use_gpu' flag is set.
Will distribute tasks to GPUs automatically.
There are not enough GPU Resources available to spawn 20 processes. Reducing number of parallel runs to 1
Using device Tesla P100-PCIE-16GB for training (cuda:0).
Forcing interval set by user: 10
Epoch 1 took 15.94s | Cumulative time (h:mm:ss): 0:00:15 | Loss = 0.9121284484863281
Epoch 2 took 15.81s | Cumulative time (h:mm:ss): 0:00:31 | Loss = 0.8989867568016052
Epoch 3 took 15.9s | Cumulative time (h:mm:ss): 0:00:47 | Loss = 0.9115966558456421
Epoch 4 took 15.64s | Cumulative time (h:mm:ss): 0:01:03 | Loss = 0.8702396154403687
Epoch 5 took 15.51s | Cumulative time (h:mm:ss): 0:01:18 | Loss = 0.8451351523399353
Epoch 6 took 15.61s | Cumulative time (h:mm:ss): 0:01:34 | Loss = 0.8590642213821411
Epoch 7 took 15.74s | Cumulative time (h:mm:ss): 0:01:50 | Loss = 0.912825882434845
Epoch 8 took 15.59s | Cumulative time (h:mm:ss): 0:02:05 | Loss = 0.9110747575759888
Epoch 9 took 15.61s | Cu

In [22]:
# --- Configuration Cell: Set 250 Epochs & 50-Epoch Metrics ---

# We must be in the experiment directory to edit the file
%cd /kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG

print("--- Modifying n_epochs ---")
# 1. Change n_epochs from [10] (or whatever it is) to [250]
!sed -i "s/Argument('n_epochs', \[[0-9]*\])/Argument('n_epochs', [250])/" ubermain.py
# Verify the change
!grep -n "n_epochs" ubermain.py

print("\n--- Modifying test_interval (for metrics) ---")
# 2. Change test_interval from its default (e.g., 10) to [50]
!sed -i "s/Argument('test_interval', \[[0-9]*\])/Argument('test_interval', [50])/" ubermain.py
# Verify the change
!grep -n "test_interval" ubermain.py

# 3. Go back to the main BPTT_TF directory
%cd /kaggle/working/dendPLRNN/BPTT_TF

print("\n✅ Configuration complete. Ready to run final training.")

/kaggle/working/dendPLRNN/BPTT_TF/Experiments/Table1/ECG
--- Modifying n_epochs ---
23:    args.append(Argument('n_epochs', [250]))

--- Modifying test_interval (for metrics) ---
/kaggle/working/dendPLRNN/BPTT_TF

✅ Configuration complete. Ready to run final training.


In [23]:
# --- Cell 3 (FINAL EXPERIMENT): Run 250 Epochs ---

!echo "Running python syntax check..."
!python -m py_compile bptt/novelties.py
!python -m py_compile bptt/bptt_algorithm.py
!echo "✅ Syntax OK."
!echo " "

!echo "Launching FINAL 250-epoch training (metrics every 50 epochs)..."

# Get the full, absolute paths for PYTHONPATH
ROOT_DIR = "/kaggle/working/dendPLRNN"
BPTT_DIR = "/kaggle/working/dendPLRNN/BPTT_TF"
FULL_PYTHONPATH = f"{BPTT_DIR}:{ROOT_DIR}"

# This command will now run your full experiment
!PYTHONPATH="{FULL_PYTHONPATH}" python -u Experiments/Table1/ECG/ubermain.py

!echo "--- Training finished ---"
!echo " "

# Find and copy the latest model
!bash -c 'LATEST_MODEL=$(find results -type f -name "*.pt" | sort | tail -n 1 || true); \
if [ -f "$LATEST_MODEL" ]; then \
  cp "$LATEST_MODEL" /kaggle/working/final_trained_model.pt; \
  echo "✅ Saved model to /kaggle/working/final_trained_model.pt"; \
else \
  echo "⚠️ No .pt checkpoint found in results/ — check the log above for errors."; \
fi'

Running python syntax check...
✅ Syntax OK.
 
Launching FINAL 250-epoch training (metrics every 50 epochs)...
'use_gpu' flag is set.
Will distribute tasks to GPUs automatically.
There are not enough GPU Resources available to spawn 20 processes. Reducing number of parallel runs to 1
Using device Tesla P100-PCIE-16GB for training (cuda:0).
Forcing interval set by user: 10
Epoch 1 took 15.66s | Cumulative time (h:mm:ss): 0:00:15 | Loss = 0.9285472631454468
Epoch 2 took 15.56s | Cumulative time (h:mm:ss): 0:00:31 | Loss = 0.8544243574142456
Epoch 3 took 15.38s | Cumulative time (h:mm:ss): 0:00:46 | Loss = 0.8121728897094727
Epoch 4 took 15.51s | Cumulative time (h:mm:ss): 0:01:02 | Loss = 0.8122896552085876
Epoch 5 took 15.41s | Cumulative time (h:mm:ss): 0:01:17 | Loss = 0.8150342702865601
Epoch 6 took 15.71s | Cumulative time (h:mm:ss): 0:01:33 | Loss = 0.7717113494873047
Epoch 7 took 15.54s | Cumulative time (h:mm:ss): 0:01:48 | Loss = 0.6030418276786804
Epoch 8 took 15.45s | Cumulativ

In [ ]:
# --- 2. Define the Code to Inject (DEDENTED) ---
insertion_block = textwrap.dedent("""
    # --- novelties (MAR / TSD / safe noise) ---

    # --- MAR ABLATED ---
    # try:
    #     mar = novelties.manifold_attractor_regularization(self.model, lambda_mar=0.01)
    #     if isinstance(mar, float) or (hasattr(mar, "item") and callable(getattr(mar, "item"))):
    #         loss = loss + (mar if isinstance(mar, float) else mar)
    # except Exception: pass

    try:
        tsd = novelties.temporal_self_distillation_from_model(self.model, weight=0.05)
        if hasattr(tsd, "item"): loss = loss + tsd
    except Exception: pass

    try:
        if "inp" in locals():
            inp = novelties.inject_latent_noise(inp, training=self.model.training if hasattr(self.model, "training") else True, noise_level=getattr(self, "noise_level", 0.02))
    except Exception: pass
""")